In [1]:
import string
import random
import pandas as pd
from typing import List
from pyspark.sql.pandas.functions import pandas_udf
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
%load_ext autoreload
%autoreload 2
%matplotlib inline

COLUMNS = ["name","age","salary"]

比较下开不开启arrow的区别

测试下来，感觉性能提升有点奇怪。有时候会快，有时候会慢。

In [2]:
def create_random_data(row_num:int)->List[tuple]:
     result = list()
     a_str = string.ascii_uppercase
     for i in range(row_num):
         random_letter = random.choice(a_str)
         result.append((random_letter,random.randint(1,row_num),random.random()))
     return result

In [3]:
def test_performance(session_factory:DemoSQLSessionFactory, n:int = 100000):
    data = create_random_data(n)
    spark_session = session_factory.build_session()
    df = spark_session.createDataFrame(data,COLUMNS).cache()
    df.toPandas().head(5)

In [4]:
session_factory_normal = DemoSQLSessionFactory(name="normal")
%time test_performance(session_factory_normal)

CPU times: user 3.09 s, sys: 50.5 ms, total: 3.14 s
Wall time: 10.2 s


In [5]:
session_factory_arrow = DemoSQLSessionFactory(name="with arraw")
session_factory_arrow.add_config("spark.sql.execution.arrow.pyspark.enabled","true")
%time test_performance(session_factory_arrow)

CPU times: user 2.77 s, sys: 19.5 ms, total: 2.78 s
Wall time: 3.29 s


常规的HelloWorld的example。
页面上面的第一个例子。本质就是生成一个新的dataframe
1. 在annotation上面列出的是新的dataframe的col和类型
2. 他会自动的把pd的转换成spark的
3. 函数应该会分批node执行。然后再汇总。因为我看到了。hello world的函数会被执行好几次

In [8]:
session_factory_arrow.add_config('spark.sql.execution.arrow.maxRecordsPerBatch',10)
spark = session_factory_arrow.build_session()
test_data = create_random_data(row_num=1000)
basic_df = spark.createDataFrame(test_data,COLUMNS)
basic_df.show()

+----+----+--------------------+
|name| age|              salary|
+----+----+--------------------+
|   H|1000|  0.6615702849667711|
|   Y| 137| 0.49502192370290155|
|   F| 306| 0.24357611005187163|
|   U| 265|   0.611489662174653|
|   O|   5| 0.19890048211207068|
|   I|  85|  0.5142452856852364|
|   L| 497|  0.8521695721702193|
|   T| 233|  0.5205520446495158|
|   M| 601|0.021362590303023565|
|   C| 509| 0.06582586205632968|
|   L| 281| 0.40850737166518314|
|   X| 958|  0.5662043110891974|
|   D| 124|0.014962613865048158|
|   H| 769|0.016595441983602077|
|   S| 330| 0.24779907845329496|
|   P| 185|  0.7396863119855275|
|   L| 214|  0.6983798071301994|
|   T| 495|  0.8820863717262004|
|   R|  40|  0.9404487883934665|
|   I| 234|  0.8859580362471405|
+----+----+--------------------+
only showing top 20 rows



In [9]:
@pandas_udf("total double")
def func(s1: pd.Series, s2: pd.Series) -> pd.DataFrame:
    print("execute")
    s3 = pd.DataFrame()
    s3['total'] = s1 + s2
    return s3
basic_df.select(func("age","salary").alias("result")).show()


+--------------------+
|              result|
+--------------------+
|[1000.6615702849667]|
| [137.4950219237029]|
| [306.2435761100519]|
| [265.6114896621747]|
|  [5.19890048211207]|
| [85.51424528568523]|
| [497.8521695721702]|
|[233.52055204464952]|
|  [601.021362590303]|
|[509.06582586205633]|
|[281.40850737166517]|
| [958.5662043110892]|
|[124.01496261386505]|
| [769.0165954419836]|
| [330.2477990784533]|
|[185.73968631198554]|
| [214.6983798071302]|
| [495.8820863717262]|
| [40.94044878839347]|
|[234.88595803624713]|
+--------------------+
only showing top 20 rows

